In [2]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch import nn 
import copy

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, precision_score, recall_score, auc
from sklearn.model_selection import KFold
torch.manual_seed(1)    # reproducible torch:2 np:3
np.random.seed(1)

from config import BIN_config_DBPE
from models import BIN_Interaction_Flat
from stream import BIN_Data_Encoder

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

/home/owen/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Select which model to use

In [3]:
import os

model_dir = r"./results/davis/20220519_131941/train/model_dir"

# Load the configurations

In [4]:
config = BIN_config_DBPE()

In [5]:
config

{'batch_size': 16,
 'input_dim_drug': 23532,
 'input_dim_target': 16693,
 'train_epoch': 50,
 'max_drug_seq': 50,
 'max_protein_seq': 545,
 'emb_size': 384,
 'dropout_rate': 0.1,
 'scale_down_ratio': 0.25,
 'growth_rate': 20,
 'transition_rate': 0.5,
 'num_dense_blocks': 4,
 'kernal_dense_size': 3,
 'intermediate_size': 1536,
 'num_attention_heads': 12,
 'attention_probs_dropout_prob': 0.1,
 'hidden_dropout_prob': 0.1,
 'flat_dim': 78192}

In [6]:
model = BIN_Interaction_Flat(**config)

model = model.cuda()

# Load the model

In [7]:
# original saved file with DataParallel
state_dict = torch.load(os.path.join(model_dir, "max-model.ckpt"))
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [8]:
model.eval()

BIN_Interaction_Flat(
  (demb): Embeddings(
    (word_embeddings): Embedding(23532, 384)
    (position_embeddings): Embedding(50, 384)
    (LayerNorm): LayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (pemb): Embeddings(
    (word_embeddings): Embedding(16693, 384)
    (position_embeddings): Embedding(545, 384)
    (LayerNorm): LayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (d_encoder): Encoder_MultipleLayers(
    (layer): ModuleList(
      (0): Encoder(
        (attention): Attention(
          (self): SelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): SelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm()
            (dropo

# Protein Embedding and Fasta Embedding

In [9]:
from stream import protein2emb_encoder, drug2emb_encoder

In [10]:
params = {
    'batch_size': 4,
      'shuffle': True,
      'num_workers': 0,
      'drop_last': True
         }

In [11]:
dataFolder = './dataset/DAVIS'
df_test = pd.read_csv(dataFolder + '/test.csv')
testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, df_test)
testing_generator = data.DataLoader(testing_set, **params)

In [12]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [21]:
model.icnn.register_forward_hook(get_activation('i_v'))

In [83]:
# Note:
# In order to check which layer makes the randomness output of model.eval(),
# we mock the same arch. of MolTrans decoder. 
class MyDecoder(nn.Sequential):
    def __init__(self, **config):
        super(MyDecoder, self).__init__()
        self.gpus = torch.cuda.device_count()
        self.batch_size = config['batch_size']
        self.max_d = config['max_drug_seq']
        self.max_p = config['max_protein_seq']
        self.dropout_rate = config['dropout_rate']
        self.flatten_dim = config['flat_dim']
        
        self.icnn = nn.Conv2d(1, 3, 3, padding = 0, dtype=torch.float32)
        self.dropout = nn.Dropout(p=self.dropout_rate) # this line is distinct to original code
        
        self.decoder = nn.Sequential(
            nn.Linear(self.flatten_dim, 512),
            nn.ReLU(True),

            nn.BatchNorm1d(512),
            nn.Linear(512, 64),
            nn.ReLU(True),

            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(True),

            #output layer
            nn.Linear(32, 1)
        )
        
    def forward(self, i):
        i_v_1 = i.view(int(self.batch_size/self.gpus), -1, self.max_d, self.max_p)
        i_v_2 = torch.sum(i_v_1, dim = 1)
        i_v_3 = torch.unsqueeze(i_v_2, 1)
        
        # Dropout ################
        # i_v_4 = F.dropout(i_v_3, p = self.dropout_rate) # where the randomness occurs
        i_v_4 = self.dropout(i_v_3) # we use nn.Dropout() instead F.dropout()
        ##########################
        
        f = self.icnn(i_v_4)
        f_1 = f.view(int(self.batch_size/self.gpus), -1)
        score = self.decoder(f_1)
        
        # return i_v_1, i_v_2, i_v_3, i_v_4, f, f_1
        return i_v_1, i_v_2, i_v_3, f, f_1, score
    
        
myDecoder = MyDecoder(**config)
myDecoder = myDecoder.cuda()
myDecoder.eval()

MyDecoder(
  (icnn): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (dropout): Dropout(p=0.1, inplace=False)
  (decoder): Sequential(
    (0): Linear(in_features=78192, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=512, out_features=64, bias=True)
    (4): ReLU(inplace=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [84]:
with torch.no_grad():
    for i, (d, p, d_mask, p_mask, label) in enumerate(testing_generator):
        model.decoder.register_forward_hook(get_activation('i_v'))
        drug, drug_mask = d.long().cuda(), d_mask.long().cuda()
        protien, protien_mask = p.long().cuda(), p_mask.long().cuda()
        score = model(drug, protien, drug_mask, protien_mask)
        
        # output_1_i_v_1, output_1_i_v_2, output_1_i_v_3, output_1_i_v_4, output_1_f, output_1_f_1 = myDecoder(score[1].cuda())
        output_1_i_v_1, output_1_i_v_2, output_1_i_v_3, output_1_f, output_1_f_1, score_11 = myDecoder(score[1].cuda())
        
        score_1 = model(drug, protien, drug_mask, protien_mask)
        # output_2_i_v_1, output_2_i_v_2, output_2_i_v_3, output_2_i_v_4, output_2_f, output_2_f_1 = myDecoder(score_1[1].cuda())
        output_2_i_v_1, output_2_i_v_2, output_2_i_v_3, output_2_f, output_2_f_1, score_22 = myDecoder(score_1[1].cuda())
        
        print(torch.eq(output_1_i_v_3, output_2_i_v_3))
        print(torch.eq(score_11, score_22))

        if i == 0: break

torch.float32
torch.float32
tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]],


        [[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]]], device='cuda:0')
tensor([[True],
        [True]], device='cuda:0')


d_v_tensor = torch.from_numpy(d_v).long().cuda().unsqueeze(0)
input_mask_d_tensor = torch.from_numpy(input_mask_d).long().cuda().unsqueeze(0)
p_v_tensor = torch.from_numpy(p_v).long().cuda().unsqueeze(0)
input_mask_p_tensor = torch.from_numpy(input_mask_p).long().cuda().unsqueeze(0)
score = model.forward(d_v_tensor, 
                      p_v_tensor, 
                      input_mask_d_tensor, 
                      input_mask_p_tensor)
print(score.shape)
m = torch.nn.Sigmoid()
logits = torch.squeeze(m(score))
logits = logits.detach().cpu().numpy()
print(logits)